In [ ]:
# from scipy.special import comb
# import pandas as pd

# def evaluate(groundtruth, parsedresult):
#     """
#     Đánh giá độ chính xác Accuracy và F1-score của thuật toán phân tích log bằng cách so sánh dữ liệu ground truth với kết quả đã phân tích.

#     Args:
#         groundtruth (str): Đường dẫn tới file ground truth (dữ liệu đúng).
#         parsedresult (str): Đường dẫn tới file kết quả đã phân tích.

#     Returns:
#         tuple: (f_measure, accuracy)
#             - f_measure (float): Chỉ số F1-score (đánh giá độ chính xác tổng hợp).
#             - accuracy (float): Độ chính xác của việc gán nhãn log.
#     """
#     df_groundtruth = pd.read_csv(groundtruth)       # File chính xác
#     df_parsedlog = pd.read_csv(parsedresult)        # File kết quả phân tích
    
#     # Remove invalid groundtruth event Ids
#     non_empty_log_ids = df_groundtruth[~df_groundtruth["EventId"].isnull()].index # Lấy index của các dòng không rỗng EventId
#     df_groundtruth = df_groundtruth.loc[non_empty_log_ids]  # Lấy các dòng log không rỗng tương ứng trong file chính xác
#     df_parsedlog = df_parsedlog.loc[non_empty_log_ids]      # Tương tự
    
#     (precision, recall, f_measure, accuracy) = get_accuracy(
#         df_groundtruth["EventId"], df_parsedlog["EventId"]
#     )
#     print(
#         "Precision: {:.4f}, Recall: {:.4f}, F1_measure: {:.4f}, Parsing_Accuracy: {:.4f}".format(
#             precision, recall, f_measure, accuracy
#         )
#     )
#     return f_measure, accuracy


# def get_accuracy(series_groundtruth: pd.Series, series_parsedlog:pd.Series, debug=True) -> tuple:
#     """
#     Tính toán các chỉ số đánh giá của thuật toán phân tích log.
#     `Chú ý`: Giá trị tính toán được xác định là số cặp có trong nhóm log được phân cụm (vì EventID có thể khác nhau giữa file truth và file parse, do đó, cần tính một chỉ số khác).
#     Chỉ số được sử dụng không phải tính độ chính xác theo mẫu mà sẽ tính theo số cặp trong nhóm đó.

#     Giả sử với n mẫu trong nhóm A, đoán đúng k mẫu, sai (n-k) mẫu.Như vậy, số cặp đoán đúng là:
#     TP = C(k, 2) + C(n-k, 2)
#     Số cặp đoán sai: k(n-k)
#     Số cặp chính xác: C(n, 2)
#     Như vậy: TP + Số cặp sai = Số cặp chính xác
        
#     Args:
#         series_groundtruth (pd.Series): Chuỗi `EventId` của dữ liệu ground truth.
#         series_parsedlog (pd.Series): Chuỗi `EventId` của kết quả phân tích.
#         debug (bool, optional): In thông tin debug khi có lỗi. Mặc định là False.

#     Returns:
#         tuple: (precision, recall, f_measure, accuracy)
#             - precision (float): Độ chính xác (Precision).
#             - recall (float): Độ bao phủ (Recall).
#             - f_measure (float): Chỉ số F1-score.
#             - accuracy (float): Độ chính xác của việc gán nhãn log.
#     """
#     series_groundtruth_valuecounts = series_groundtruth.value_counts()
#     real_pairs = 0
#     for count in series_groundtruth_valuecounts:
#         if count > 1:
#             real_pairs += comb(count, 2)

#     series_parsedlog_valuecounts = series_parsedlog.value_counts()
#     parsed_pairs = 0
#     for count in series_parsedlog_valuecounts:
#         if count > 1:
#             parsed_pairs += comb(count, 2)

#     accurate_pairs = 0
#     accurate_events = 0  # determine how many lines are correctly parsed
#     for parsed_eventId in series_parsedlog_valuecounts.index:
#         logIds = series_parsedlog[series_parsedlog == parsed_eventId].index
#         series_groundtruth_logId_valuecounts = series_groundtruth[logIds].value_counts()
#         error_eventIds = (
#             parsed_eventId,
#             series_groundtruth_logId_valuecounts.index.tolist(),
#         )
#         error = True
#         if series_groundtruth_logId_valuecounts.size == 1:
#             groundtruth_eventId = series_groundtruth_logId_valuecounts.index[0]
#             if (
#                 logIds.size
#                 == series_groundtruth[series_groundtruth == groundtruth_eventId].size
#             ):
#                 accurate_events += logIds.size
#                 error = False
#         if error and debug:
#             print(
#                 "(parsed_eventId, groundtruth_eventId) =",
#                 error_eventIds,
#                 "failed",
#                 logIds.size,
#                 "messages",
#             )
#         for count in series_groundtruth_logId_valuecounts:
#             if count > 1:
#                 accurate_pairs += comb(count, 2)

#     precision = float(accurate_pairs) / parsed_pairs
#     recall = float(accurate_pairs) / real_pairs
#     f_measure = 2 * precision * recall / (precision + recall)
#     accuracy = float(accurate_events) / series_groundtruth.size
#     return precision, recall, f_measure, accuracy


In [ ]:
import pandas as pd
import numpy as np
import os
import sys

import Drain_setting
sys.path.append("../../parser")

from utils import evaluator

import root.Drain.Drain
import importlib

importlib.reload(root.Drain.Drain)  # Reload module
LogParser = root.Drain.Drain.LogParser  # Truy cập class LogParser


assign_dataset = ["HDFS", "BGL"]
benchmark_settings = {}
for temp in assign_dataset:
    benchmark_settings[temp] = Drain_setting.BENCHMARK_DRAIN_SETTING[temp]
input_dir = '../../data_benchmark/loghub_2k/'
output_dir = '../result_data/Drain/'


bechmark_result = []
for dataset, setting in benchmark_settings.items():
    print("\n======================== Evaluation on %s ========================" % dataset)
    indir = os.path.join(input_dir, os.path.dirname(setting["log_file"]))
    log_file = os.path.basename(setting["log_file"])

    parser = LogParser(
        log_format=setting["log_format"],
        indir=indir,
        outdir=output_dir,
        rex=setting["regex"],
        depth=setting["depth"],
        st=setting["st"],
    )
    parser.parse(log_file)

    F1_measure, accuracy = evaluator.evaluate(
        groundtruth=os.path.join(indir, log_file + "_structured.csv"),
        parsedresult=os.path.join(output_dir, log_file + "_structured.csv"),
    )
    bechmark_result.append([dataset, F1_measure, accuracy])


print("\n======================== Overall evaluation results ========================")
df_result = pd.DataFrame(bechmark_result, columns=["Dataset", "F1_measure", "Accuracy"])
df_result.set_index("Dataset", inplace=True)
print(df_result)
df_result.to_csv("Drain_bechmark_result.csv", float_format="%.6f")



======================== Evaluation on HDFS ========================
Parsing file: ../../data_benchmark/loghub_2k/HDFS\HDFS_2k.log
Total lines:  2000
Processed 100.0% of log lines.
Parsing done. [Time taken: 0:00:00.186829]
(parsed_eventId, groundtruth_eventId) = ('04137b95', ['E4']) failed 2 messages
(parsed_eventId, groundtruth_eventId) = ('98012f03', ['E4']) failed 2 messages
(parsed_eventId, groundtruth_eventId) = ('415a1760', ['E4']) failed 1 messages
Precision: 1.0000, Recall: 1.0000, F1_measure: 1.0000, Parsing_Accuracy: 0.9975

======================== Evaluation on BGL ========================
Parsing file: ../../data_benchmark/loghub_2k/BGL\BGL_2k.log
Total lines:  2000
Processed 100.0% of log lines.
Parsing done. [Time taken: 0:00:00.170242]
(parsed_eventId, groundtruth_eventId) = ('cc8ff6de', ['E101', 'E103', 'E102', 'E105', 'E104']) failed 21 messages
(parsed_eventId, groundtruth_eventId) = ('ad69fd32', ['E32', 'E30']) failed 12 messages
(parsed_eventId, groundtruth_event